# 💬 Week 07-08 · Notebook 02 · Message Structures & Conversational Safety

Craft resilient multi-turn conversations using LangChain message objects while satisfying bilingual support and compliance checks.

## 🎯 Learning Objectives
- Differentiate `SystemMessage`, `HumanMessage`, `AIMessage`, and `FunctionMessage` in manufacturing workflows.
- Apply safety disclaimers and language toggles consistently across conversations.
- Persist conversational state for shift hand-offs with audit-ready metadata.
- Simulate bilingual dialogue for cross-plant collaboration.

## 🧩 Scenario
Technicians in Pune (English) and Monterrey (Spanish) collaborate on a spindle vibration issue. Messages must remain in policy-compliant format, capturing translation provenance for post-mortems.

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

system_msg = SystemMessage(content='Eres un asistente bilingüe que responde en el idioma del técnico. Siempre incluye advertencias de seguridad y referencias a SOP.')
human_msg_en = HumanMessage(content='Press 7 is overheating after the lubricant swap. What should we check?')
ai_response_en = AIMessage(content='Step 1: Verify coolant flow... (SOP-122 §3.1). Safety: Lockout-tagout before inspection.')
human_msg_es = HumanMessage(content='Los sensores marcan temperatura alta en el husillo. Hay pasos adicionales?')

conversation = [system_msg, human_msg_en, ai_response_en, human_msg_es]

### 🗣️ Language Detection & Switching
Use LangChain callbacks or middleware to detect language and switch prompts dynamically.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# A simple language detection heuristic for the example
def detect_language(text: str) -> str:
    """
    Mock language detection. In a real application, use a robust library 
    like langdetect or a dedicated API.
    """
    spanish_indicators = ['los', 'en', 'hay', 'un', 'una', 'temperatura', 'alta']
    if any(word in text.lower() for word in spanish_indicators):
        return 'es'
    return 'en'

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.3)

# Get the last human message from the conversation history
last_human_message = human_msg_es

# Detect language and set the appropriate system prompt
language = detect_language(last_human_message.content)
if language == 'es':
    prompt_prefix = SystemMessage(content='Responde en español con pasos accionables y advertencias de seguridad.')
else:
    prompt_prefix = SystemMessage(content='Respond in English with clear steps and safety warnings.')

# Construct the message list for the LLM, including the dynamic system prompt
messages_for_llm = conversation + [prompt_prefix]

# Invoke the model
completion = llm.invoke(messages_for_llm)

print(f"Detected Language: '{language}'")
print("---")
print(f"AI Response:\n{completion.content}")

## 🧾 Conversation Log Schema
| Field | Description | Required |
- Store logs in encrypted storage with retention policy (7 years per legal).

## 🧪 Lab Assignment
1. Implement a conversational buffer that trims history beyond 12 turns while preserving key instructions.
2. Add translation provenance metadata (`translated_from`) when language switches.
3. Generate a bilingual escalation message for EHS incidents requiring supervisor approval.
4. Validate conversation logs against the schema and export to JSONL for audits.

## ✅ Checklist
- [ ] Bilingual conversation flow implemented
- [ ] Language detection controls in place
- [ ] Conversation logs schema enforced
- [ ] Lab deliverables reviewed with SMEs

## 📚 References
- LangChain Message Reference
- ISO 45001 Safety Communication Standards
- Week 05-06 Governance Utilities